### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
import locale


# Set number and currency formatting
locale.setlocale(locale.LC_ALL,'en_US')

# File to Load (Remember to Change These)
schools_data = os.path.join('Resources', 'schools_complete.csv')
student_data = os.path.join('Resources', 'students_complete.csv')

# Read School and Student Data File and store into Pandas DataFrames
schools_df = pd.read_csv(schools_data)
students_df = pd.read_csv(student_data)

# Combine the data into a single dataset.  
schools_df.rename(columns = {'name':'school_name'}, inplace = True)

merged_df = students_df.merge(schools_df, how = 'left', on = 'school_name')

merged_df.head


<bound method NDFrame.head of        Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  
0                 66  

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [8]:
#create array of unique school names
unique_school_names = schools_df['school_name'].unique()

#gives the length of unique school names to give us how many schools
school_count = len(unique_school_names)

#district student count
dist_student_count = schools_df['size'].sum()

#student count from student file (to verify with district student count)
total_student_rec = students_df['student_name'].count()

#total budget
total_budget = schools_df['budget'].sum()

#calculations for number and % passing reading
num_passing_reading = students_df.loc[students_df['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

#calculations for number and % passing math
num_passing_math = students_df.loc[students_df['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec

#average math score calculation
avg_math_score = students_df['math_score'].mean()

#average reading score calculation
avg_reading_score = students_df['reading_score'].mean()

#Overall Passing Rate Calculations
overall_pass = students_df[(students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70)]['student_name'].count()/total_student_rec

# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate']]

# Create a dataframe to hold the above results

results = pd.DataFrame({"Total Schools":[school_count],"Total Students":[dist_student_count],"Total Budget":[total_budget],"Average Math Score":[avg_math_score],"Average Reading Score":[avg_reading_score],"% Passing Math":[perc_pass_math],"% Passing Reading":[perc_pass_reading], "% Overall Passing":[overall_pass]})


#format cells
results['Total Students'] = results['Total Students'].map("{:,}".format)
results['Total Budget'] = results['Total Budget'].map("${:,.2f}".format)
results['Average Math Score'] = results['Average Math Score'].map("{:.2f}".format)
results['Average Reading Score'] = results['Average Reading Score'].map("{:.2f}".format)
results['% Passing Math'] = results['% Passing Math'].map("{:.2f}".format)
results['% Passing Reading'] = results['% Passing Reading'].map("{:.2f}".format)
results['% Overall Passing'] = results['% Overall Passing'].map("{:.2f}".format)

dist_sum

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,24649428,81.87784,78.985371,0.858055,0.749809,0.651723


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [16]:
#groups by school
by_school = merged_df.set_index('school_name').groupby(['school_name'])

#school types
sch_types = schools_df.set_index('school_name')['type']

# total students by school
stu_per_sch = by_school['Student ID'].count()

# school budget
sch_budget = schools_df.set_index('school_name')['budget']

#per student budget
stu_budget = schools_df.set_index('school_name')['budget']/schools_df.set_index('school_name')['size']

#avg scores by school
avg_math = by_school['math_score'].mean()
avg_read = by_school['reading_score'].mean()

# % passing scores
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch 

sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})




#munging
sch_summary = sch_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#formatting
sch_summary = sch_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget", 
                                         "Average Math Score", "Average Reading Score", "% Passing Math", 
                                         "% Passing Reading", "Overall Passing Rate"]]

sch_summary["Total School Budget"] = sch_summary["Total School Budget"].map("$(:,2f)".format)
sch_summary



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$(:,2f)",628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,Charter,1858,"$(:,2f)",582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Figueroa High School,District,2949,"$(:,2f)",639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Ford High School,District,2739,"$(:,2f)",644.0,77.102592,80.746258,0.683096,0.792990,0.542899
Griffin High School,Charter,1468,"$(:,2f)",625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Hernandez High School,District,4635,"$(:,2f)",652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Holden High School,Charter,427,"$(:,2f)",581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Huang High School,District,2917,"$(:,2f)",655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Johnson High School,District,4761,"$(:,2f)",650.0,77.072464,80.966394,0.660576,0.812224,0.535392
Pena High School,Charter,962,"$(:,2f)",609.0,83.839917,84.044699,0.945946,0.959459,0.905405


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [18]:
# Top Performing Schools (By % Overall Passing)
top_performing = sch_summary.sort_values(["Overall Passing Rate"], ascending = False)
top_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$(:,2f)",582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,"$(:,2f)",638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,"$(:,2f)",625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,"$(:,2f)",578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,"$(:,2f)",609.0,83.839917,84.044699,0.945946,0.959459,0.905405


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [19]:
# Bottom Performing Schools (By % Overall Passing)
bottom_performing = sch_summary.sort_values(["Overall Passing Rate"], ascending = True)
bottom_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$(:,2f)",637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,"$(:,2f)",639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,"$(:,2f)",655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,"$(:,2f)",652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,"$(:,2f)",650.0,77.072464,80.966394,0.660576,0.812224,0.535392


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [23]:
#creates grade level average math scores for each school 
ninth_math = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

#show and format
math_scores

,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [24]:
#creates grade level average reading scores for each school
ninth_reading = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#merges the reading score averages by school and grade together
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

reading_scores

,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [26]:
# create spending bins
bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ['< $585', "$585 - 614", "$615 - 644", "> $644"]
merged_df['spending_bins'] = pd.cut(merged_df['budget']/merged_df['size'], bins, labels = group_name)

#group by spending
by_spending = merged_df.groupby('spending_bins')

#calculations
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()

            
# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_name)

scores_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
< $585,83.363065,83.964039,0.937029,0.966866,0.906407
$585 - 614,83.529196,83.838414,0.941241,0.958869,0.901212
$615 - 644,78.061635,81.434088,0.714004,0.836148,0.602893
> $644,77.049297,81.005604,0.662308,0.811094,0.535288


## Scores by School Size

* Perform the same operations as above, based on school size.

In [27]:
# create size bins
bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
merged_df['size_bins'] = pd.cut(merged_df['size'], bins, labels = group_name)

#group by spending
by_size = merged_df.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

            
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.828654,83.828654,0.939525,0.960403,0.901368
Medium (1000-2000),83.372682,83.372682,0.936165,0.967731,0.906243
Large (>2000),77.477597,77.477597,0.686524,0.821252,0.565740


## Scores by School Type

* Perform the same operations as above, based on school type

In [28]:
# group by type of school
by_type = merged_df.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.406183,83.406183,0.937018,0.966459,0.905609
District,76.987026,76.987026,0.665184,0.809052,0.536959


In [ ]:
#Analysis
    #1. The increase in budget which is typically tied to students falling into certain high risk categories does not postively impact overall passing rates at schools. Schools need to look at additional data to determine what can improve student outcomes
    #2.Charter schools seem to have a much better overall passing rate in PyCity. It would be interesting to look at other statistics to determine why this may be the case (i.e. characteristics of student population at charter schools vs district, characteristics of teacher population at charter schools vs district)
    #3. Being a larger school (over 2000 students) contributes significantly to overall passing rates. 35% less students pass both reading and math in schools with over 2000 students.